In [1]:
#from yelp.client import Client
import requests
import pprint
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql

In [2]:
def search_business(offset):
    api_key = "OnCmW229lglON7t4NBgK55sdvTzDfUIq-AJYVQDPnbSLwbS3rC8ky_dPBk6GETFAKBjrUfYYqBIgo5VN1G6Eu51-_gL3si3cNKJzZTxAfaiKf2YazE_1dBnkpyTuW3Yx" #  Replace this with your real API key
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

    url_params = {
                'term': "Restaurants",
                'location': "Washington+DC",
                'limit': 50,
                'total': offset
                }
    
    response = requests.get(url, headers=headers, params=url_params)
    return response.json() # Returns a JSON.


In [5]:
if __name__ == "__main__":
    location = []
    for offset_num in np.arange(50,1000,50) :
        output_json = search_business(offset_num)
        for x in range(50):
            y = output_json['businesses'][x]['location']['display_address']
            #print(y)
            y = " ".join(y)
            location.append(y) 
# I want up to 550 results, in steps of 50 results per request.
        try:
            output_json = search_business(offset_num) # Executing the function defined above.
            #print(offset_num) # Making sure each offset iteration is running
            #print(output_json) # If you wanna check the JSON for each iteration
          
            if offset_num == 50:
                df_first = pd.DataFrame.from_dict(output_json['businesses'])
                        
# 'businesses' because that's the primary key of the JSON (i.e. pull all attribute data by calling 
# that one key). This is something you can figure out reading the API documentation or visually
# parsing the JSON. 
            else:
                df2 = pd.DataFrame.from_dict(output_json['businesses'])
                df_first = df_first.append(df2)
                df_final = df_first[['name', 'rating']].copy()
                df_final["Address"] = location
# The conditional statement above is so that I can append my results into a single dataframe, to 
# save into a single csv document.
        except AttributeError:
            print("error at ", offset_num) # Helpful for debugging purposes
            

In [64]:
#df_first.head(100)
#print(len(df_first['name']))


In [15]:
df_final = df_final.rename(columns={"name": "RestaurantName", "rating": "Rating"})
df_final.head()
print(len(df_final['RestaurantName']))

950


In [9]:
rds_connection_string = "root:boston@127.0.0.1/restaurants_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [88]:
x = df_final["Rating"][1]
print (x)

1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
1    4.0
Name: Rating, dtype: float64


In [10]:
engine.table_names()

['google_info', 'yelp_info']

In [11]:
df_final.to_sql(name='yelp_info', con=engine, if_exists='append', index=False)

In [12]:
pd.read_sql_query('select * from yelp_info', con=engine).head()

,id,RestaurantName,Rating,Address
0,1,Founding Farmers - Washington,4,"1924 Pennsylvania Ave NW Washington, DC 20006"
1,2,Old Ebbitt Grill,4,"675 15th St NW Washington, DC 20005"
2,3,The Pig,4,"1320 14th St NW Washington, DC 20005"
3,4,Farmbird,5,"625A H St NE Washington, DC 20002"
4,5,Unconventional Diner,4.5,"1207 9th St NW Washington, DC 20001"
